Running benchmark in notebook for debug.


In [2]:
import os
import matplotlib.pyplot as plt
BASE_PATH = os.path.abspath('')
print(BASE_PATH)

/home/turtlewizard/thesis-mppi-model-ident/workspace/src/controller_optimization


In [3]:
# create mockup logger
class Logger:
    def info(self, msg: str) -> None:
        print('Info: ' + msg)

    def error(self, msg: str) -> None:
        print('Error: ' + msg)

    def debug(self, msg: str) -> None:
        print('Debug: ' + msg)

    def isEnabledFor(self, *args, **kwargs) -> bool:
        return True


logger = Logger()
logger.info('Hello from mockup logger.')

Info: Hello from mockup logger.


In [4]:
from controller_benchmark import ControllerBenchmark

controller_benchmark = ControllerBenchmark(
    logger=logger,
    config_path=os.path.join(
        BASE_PATH, 'config/controller_benchmark_config.yaml'),
    base_path=BASE_PATH
)

controller_benchmark.launch_nodes()

Info: Loading config...
Debug: Map config file: /home/turtlewizard/thesis-mppi-model-ident/workspace/src/controller_optimization/config/controller_benchmark_config.yaml
Debug: Config: 
 {'cmd_vel_topic': '/cmd_vel',
 'controller': 'FollowPathMPPI',
 'corridor_empty': {'goal_pose': {'x': 2.0, 'y': 0.0, 'yaw': 0.0},
                    'path': 'maps/corridor_empty.yaml',
                    'start_pose': {'x': 0.0, 'y': 0.0, 'yaw': 0.0}},
 'corridor_mid_obstacles': {'goal_pose': {'x': 2.0, 'y': 0.0, 'yaw': 0.0},
                            'path': 'maps/corridor_mid_obstacles.yaml',
                            'start_pose': {'x': 0.0, 'y': 0.0, 'yaw': 0.0}},
 'corridor_mixed': {'goal_pose': {'x': 8.25, 'y': 0.0, 'yaw': 0.0},
                    'path': 'maps/corridor_mixed.yaml',
                    'start_pose': {'x': 0.0, 'y': 0.0, 'yaw': 0.0}},
 'corridor_side_obstacles': {'goal_pose': {'x': 2.0, 'y': 0.0, 'yaw': 0.0},
                             'path': 'maps/corridor_side_obstacles

[INFO] [1730323699.604569420] [parameter_manager]: ParameterManager for controller_server initialized
[INFO] [1730323699.678033973] [gazebo_interface]: Gazebo interface initialized
[INFO] [1730323699.681481023] [odom_subscriber]: odom_subscriber initialized
[INFO] [1730323699.685033521] [cmd_vel_subscriber]: cmd_vel_subscriber initialized
[INFO] [1730323699.690402059] [mppi_critic_subscriber]: mppi_critic_subscriber initialized


In [6]:
# run benchmark with default parameters
controller_benchmark.run_benchmark()

Info: Running benchmark...


[INFO] [1730323713.570174414] [parameter_manager]: Setting parameters for controller_server.


Info: Starting data collection.
Info: Changing map to: /home/turtlewizard/thesis-mppi-model-ident/workspace/src/controller_optimization/maps/corridor_mixed.yaml


[INFO] [1730323714.078625623] [basic_navigator]: Change map request was successful!
[INFO] [1730323714.597391481] [basic_navigator]: Publishing Initial Pose


Info: Getting global plan for corridor_mixed with planner: GridBased.
Info: ___ Starting controller: FollowPathMPPI, on map: corridor_mixed. ___


[INFO] [1730323715.107217524] [basic_navigator]: Getting path...
[INFO] [1730323715.139131642] [basic_navigator]: Executing path...


Info: ___ Controller FollowPathMPPI on map corridor_mixed finished in 30.896062247000003 [s]. ___
Debug: Collecting results between 1730323715.1385353 and 1730323746.0345974.
Info: Stopping data collection.
Info: Benchmark finished in 32.4690 seconds.


(True,
 [ControllerResult(controller_name='FollowPathMPPI', map_name='corridor_mixed', start_time=1730323715138535244, time_elapsed=30896062247, success=True, path_xy=array([[ 1.49011612e-08,  2.23517418e-08],
         [ 4.59343112e-02, -8.27480865e-04],
         [ 7.09313552e-02, -1.21400507e-03],
         [ 9.59261104e-02, -1.72708186e-03],
         [ 1.20918005e-01, -2.36442242e-03],
         [ 1.45905130e-01, -3.16703473e-03],
         [ 1.70886534e-01, -4.12976897e-03],
         [ 1.95859259e-01, -5.29716172e-03],
         [ 2.20822066e-01, -6.65977160e-03],
         [ 2.45770473e-01, -8.26518697e-03],
         [ 2.70703144e-01, -1.00989120e-02],
         [ 2.95613785e-01, -1.22109191e-02],
         [ 3.20070477e-01, -1.73945206e-02],
         [ 3.44819089e-01, -2.09309358e-02],
         [ 3.69796964e-01, -2.19820756e-02],
         [ 3.94769022e-01, -2.31653947e-02],
         [ 4.19767687e-01, -2.34158296e-02],
         [ 4.44764540e-01, -2.38182802e-02],
         [ 4.69764350e-01

In [9]:
import numpy as np
from itertools import product
# Generate a list of weights from 1-100
n = 10
weights = np.linspace(1.0, 100.0, n)
critics = ['ConstraintCritic', 'GoalCritic', 'PreferForwardCritic',
           'CostCritic', 'PathAlignCritic', 'PathFollowCritic', 'PathAngleCritic']

# weights = [1.0, 10.0, 200.0]
critics = ['CostCritic', 'PathFollowCritic']

parameter_space = list(product(weights, repeat=len(critics)))
print(f'Possible parameter combinations: {len(parameter_space)}')

i = 0
results: list[dict[int, bool]] = []
critic_weights = []
for weights in parameter_space:
    params = {critic: weight for critic, weight in zip(critics, weights)}
    critic_weights.append(params)

    success, res = controller_benchmark.run_benchmark(parameters=params, store_results=False)
    msg = 'successful' if success else 'unsuccessful'
    print(f'Iteration {i} was {msg}. with weights {weights}')

    controller_benchmark.save_result(res[0], sub_folder='paramspace0')

    results.append({i: success})
    i += 1

Possible parameter combinations: 100
Info: Running benchmark...
Info: Starting data collection.
Info: Changing map to: /home/turtlewizard/thesis-mppi-model-ident/workspace/src/controller_optimization/maps/corridor_mixed.yaml


[INFO] [1730324129.712874068] [parameter_manager]: Setting parameters for controller_server.
[INFO] [1730324129.736488763] [basic_navigator]: Change map request was successful!
[INFO] [1730324130.261933772] [basic_navigator]: Publishing Initial Pose


Info: Getting global plan for corridor_mixed with planner: GridBased.


[INFO] [1730324130.774151216] [basic_navigator]: Getting path...


Info: ___ Starting controller: FollowPathMPPI, on map: corridor_mixed. ___


[INFO] [1730324131.056248765] [basic_navigator]: Executing path...


Info: ___ Controller FollowPathMPPI on map corridor_mixed finished in 30.891491056000003 [s]. ___
Debug: Collecting results between 1730324131.0554202 and 1730324161.9469113.
Info: Stopping data collection.
Info: Benchmark finished in 32.2429 seconds.
Iteration 0 was successful. with weights (1.0, 1.0)
Info: Saving result: corridor_mixed.


FileNotFoundError: [Errno 2] No such file or directory: '/home/turtlewizard/thesis-mppi-model-ident/workspace/src/controller_optimization/results/paramspace0/controller_benchmark_result_2024-10-30-21-36.pickle'

In [12]:
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV
import copy

# Generate a list of weights from 1-100
n = 10
weights = np.linspace(1.0, 100.0, n)

critics = ['ConstraintCritic', 'GoalCritic', 'PreferForwardCritic',
           'CostCritic', 'PathAlignCritic', 'PathFollowCritic', 'PathAngleCritic']

critics = ['CostCritic', 'PathFollowCritic']

# Create the parameter grid
param_grid = {}
for critic in critics:
    param_grid[critic + '.cost_weight'] = copy.deepcopy(weights)


def evaluate_mppi_config(params):
    controller_benchmark.run_benchmark(params)

    # Run the simulation and get performance metrics
    metric = controller_benchmark.calculate_metric()

    # Calculate a performance score
    performance_score = metric.ms_linear_jerk * 5.0 + metric.ms_angular_jerk

    return performance_score  # Lower score indicates better performance


class MPPIControllerOptimizer(BaseEstimator):
    def __init__(self):
        self.params = controller_benchmark.default_controller_params

    def fit(self, X, y=None):
        # This method is required but does not need to do anything for grid search
        return self

    def set_params(self, **params):
        # Update parameters with the new values from grid search
        for critic in critics:
            self.params[critic + '.cost_weight'] = params[critic + '.cost_weight']
        return self

    def score(self, X, y=None):
        # Call the evaluation function with the parameters
        return -evaluate_mppi_config(self.params)  # Negate for minimization


# Instantiate the optimizer and perform the grid search
optimizer = MPPIControllerOptimizer()
grid_search = GridSearchCV(
    estimator=optimizer,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # Use a scoring method suitable for your evaluation
    cv=2,  # Change this to at least 2
    verbose=1
)

# Fit the grid search (the input X can be any valid input; it's not used here)
X_dummy = np.random.rand(3, 1)
grid_search.fit(X=X_dummy)  # Dummy input, as we only need parameter search here

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found:", best_params)

Fitting 2 folds for each of 100 candidates, totalling 200 fits
Best parameters found: {'CostCritic.cost_weight': 1.0, 'PathFollowCritic.cost_weight': 1.0}


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 969, in _score
    scores = scorer(estimator, X_test, **score_params)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:1103: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan